<a href="https://colab.research.google.com/github/vramius1017/Ceps/blob/main/inca_flask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install flask-ngrok
!pip install flask-bootstrap

In [12]:
!pip install biopython

     |████████████████████████████████| 2.3 MB 5.1 MB/s 


In [ ]:
# outil clean de chaîne de caracteres

def clean_data(x):
    chaine = str(x)
    chaine = chaine.replace("['"," ")
    chaine = chaine.replace("']"," ")
    chaine = chaine.replace("'"," ")
    return chaine

In [ ]:
from Bio import Entrez 
from Bio import Medline
from ..Clean import factor
import urllib, urllib.request, urllib.error

# def ncbi_conn():
#     Entrez.api_key = "12b30395a61ef72be179af0076899757"
#     Entrez.email = "sachatorres974@gmail.com"
#     Entrez.tool = "inca" 
    
    
def Query():
    p = "green tea and neoplasms"
    pq = p + " "+"and Cochrane Database Syst Rev"
    return pq

def MetaList(q):
    # ncbi_conn()
    Entrez.email = "sachatorres974@gmail.com"
    hend = Entrez.esearch(db="pubmed",term=q)
    ret = Entrez.read(hend)
    return ret["IdList"]

def MetaCount(q):
    #ncbi_conn()
    Entrez.email = "sachatorres974@gmail.com"
    hand = Entrez.esearch(db="pubmed",term=q)
    rec = Entrez.read(hand)
    return rec["Count"]

def FetchIdList(ids):
    # ncbi_conn()
    Entrez.email = "sachatorres974@gmail.com"
    handle = Entrez.efetch(db="pubmed",id=ids,rettype="medline",retmode="text")
    record = Medline.parse(handle)
    return record
    #code jinja pour la liste des données dans le dictionnaire
    #for re in record:
    #    titre : re["TI"]
    #   r = clean_data(re["FAU"])
    #    auteurs : r

def Efetch(idl):
    # ncbi_conn()
    for re in FetchIdList(idl):
        print("Title : ",re["TI"])
        print("PMID : ",re["PMID"]+", ",end="")
        print("NLM identifier : ",re["JID"]+", ",end="")
        try:
             print("other identifier : ",re["OID"],end="")
        except KeyError :
             print("Other identifier : none ")
        r = clean_data(re["FAU"])
        print("Authors : ",r)
        mh = clean_data(re["MH"])
        print("MesH terms:")
        print(mh)
        try :
            print("Key Words: ",re["OT"])
        except KeyError:
            print("Key Words: none ")
        try:
            print("Publication source :","  ",re["SO"]) 
        except KeyError:
            print("Publication source : none")
        dt = clean_data(re["CRDT"])    
        print("Create Date : ",dt)
        revdt = clean_date(re["LR"])
        print("Revision Date :",revdt)
        print(" ")

In [16]:
# main app inca

In [15]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template , request 
import os
from google.colab import drive
drive.mount('/content/testapp')
CON_FOLDER = os.path.join('content/testapp','Connexion')
app = Flask(__name__,template_folder='/content/testapp/static')
app.config['UPLOAD_FOLDER'] = CON_FOLDER
run_with_ngrok(app)


from flask import Flask,render_template,request,url_for
import json
from Bio import Entrez
app = Flask(__name__)
#app.config.from_object('Config')

@app.route("/")
def index():
    links=["www.inmcancer.fr","www.niri.fr","www.nishare.fr"]    
    return render_template('index.html', links=links)

@app.route("/selection")
def selection():
    query = str(Query())
    count = MetaCount(query)
    return render_template('search.html', query=query, count=count)#,results=results)

@app.route("/selection_result")
def selection_result():
    query = str(Query())
    list_ids = MetaList(query)
    results=FetchIdList(list_ids)
    return render_template('search_result.html',list_ids=list_ids,results=results)

@app.route("/proposition/")
def proposition():
    return render_template('proposition.html')

@app.route("/validation/")
def validate():           
    return render_template('validate.html')

@app.route("/divers/")
def   divers():
    return  render_template('divers.html')


if __name__ == '__main__':
  app.run()

Drive already mounted at /content/testapp; to attempt to forcibly remount, call drive.mount("/content/testapp", force_remount=True).


ModuleNotFoundError: ignored